<a href="https://www.kaggle.com/code/amirmotefaker/recession-analysis?scriptVersionId=132976506" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

- A recession is an economic situation that arrives when the circulation of money in the economy is low for two consecutive quarters. When the circulation of money is low, it means people are not spending money in the market. When people don’t spend money, businesses face losses, which results in an economic slowdown and layoffs, which you must have already heard about in 2023. 

# Recession Analysis

- Recession is calculated and analyzed according to the growth in GDP, the growth in the unemployment rate, and the growth in consumer spending rate. But the most common way of measuring recession is by analyzing the monthly GDP growth data.

- So, for the task of Recession analysis, we need to have a dataset of the monthly GDP growth of a country. I found an ideal dataset for this task that is based on the monthly GDP growth rate of the United Kingdom.

# Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

# Read Data

In [2]:
data = pd.read_csv('/kaggle/input/uk-monthly-gdp/UK_monthly_gdp.csv')


In [3]:
print(data.head())

  Time Period  GDP Growth
0    /01/2020         0.3
1    /02/2020        -0.5
2    /03/2020        -7.0
3    /04/2020       -20.9
4    /05/2020         3.2


# GDP Growth

- look at the GDP growth over time

In [4]:
fig = go.Figure(data=go.Heatmap(
                   z=[data['GDP Growth']],
                   x=data.index,
                   y=['GDP Growth'],
                   colorscale='Viridis'))

fig.update_layout(title='GDP Growth over Time',
                  xaxis_title='Time Period',
                  yaxis_title='')

fig.show()

# GDP Growth Monthly

- As a recession means the decline in the circulation of money for two consecutive quarters, I will convert our monthly data into quarterly data to analyze the recession:


In [5]:
# Convert monthly data to quarterly data using resample method
data['Time Period'] = pd.to_datetime(data['Time Period'], format='/%m/%Y')
data.set_index('Time Period', inplace=True)
quarterly_data = data.resample('Q').mean()
print(quarterly_data.head())

             GDP Growth
Time Period            
2020-03-31    -2.400000
2020-06-30    -2.900000
2020-09-30     3.500000
2020-12-31     0.200000
2021-03-31     0.033333
